# 2024/04/16 Error Handling
_Author: Izaac Molina and Tyler Coles_

Details additional exceptions implemented to catch common IPM simulation errors

# Error Handling

There are (at least) three error cases we're interested in trapping --
1. something causes an IPM expression to go to zero or a negative value and results in a poisson draw with a lambda <= 0, and
2. something causes the population of a node to go to zero (when a divisor isn't "protected" by a construction like Max(1, ...)) causing a divide by zero.
3. similar to issue 1, a negative parameter can result in a negative probability in fork definitions

## First, we define ipm functions for testing purposes

In [1]:
import numpy as np

from epymorph import *
from epymorph.compartment_model import (CompartmentModel, compartment,
                                        create_model, create_symbols, edge,
                                        fork, param)
from epymorph.data_shape import Shapes
from epymorph.params import ParamValue


def load_ipm() -> CompartmentModel:
    """Load the 'sirs' IPM."""
    symbols = create_symbols(
        compartments=[
            compartment('S'),
            compartment('I'),
            compartment('R'),
        ],
        attributes=[
            param('beta', shape=Shapes.TxN),  # infectivity
            param('gamma', shape=Shapes.TxN),  # progression from infected to recovered
            param('xi', shape=Shapes.TxN)  # progression from recovered to susceptible
        ])

    [S, I, R] = symbols.compartment_symbols
    [β, γ, ξ] = symbols.attribute_symbols

    # LOOK!
    # N is NOT protected by Max(1, ...) here
    # This isn't necessary for Case 1, but is necessary for Case 2.
    N = S + I + R

    return create_model(
        symbols=symbols,
        transitions=[
            edge(S, I, rate=β * S * I / N),
            edge(I, R, rate=γ * I),
            edge(R, S, rate=ξ * R)
        ])

In [3]:
def load_ipm_fork() -> CompartmentModel:
    """Load the 'sirs' IPM."""
    symbols = create_symbols(
        compartments=[
            compartment('S'),
            compartment('I'),
            compartment('R'),
        ],
        attributes=[
            param('beta', shape=Shapes.TxN),  # infectivity
            param('gamma', shape=Shapes.TxN),  # progression from infected to recovered
            param('xi', shape=Shapes.TxN),  # progression from recovered to susceptible
            param('prob1', shape=Shapes.TxN),
            param('prob2', shape=Shapes.TxN)
        ])

    [S, I, R] = symbols.compartment_symbols
    [β, γ, ξ, prob1, prob2] = symbols.attribute_symbols

    # LOOK!
    # N is NOT protected by Max(1, ...) here
    # This isn't necessary for Case 1, but is necessary for Case 2.
    N = S + I + R

    return create_model(
        symbols=symbols,
        transitions=[
            fork(
                edge(S, I, rate=(β * S * I / N) * prob1),
                edge(S, R, rate=(β * S * I / N) * prob2),
            ),
            edge(I, R, rate=γ * I),
            edge(R, S, rate=ξ * R)
        ])

## Case 1:

If we just provide a negative value to beta in a classic SIRS model, this causes the first error.

To handle this, the exception type `IpmSimLesstThanZeroException` is used in `ipm_exec.py` to catch the less than zero error.

In [4]:
from functools import partial

from epymorph.initializer import single_location

pei_geo = geo_library['pei']()

sim = StandardSimulation(
    geo=pei_geo,
    ipm=load_ipm(),
    mm=mm_library['no'](),
    params={
        'phi': 40.0,
        'beta': 0.4,
        'gamma': -1 / 5,  # NEGATIVE NUMBER! WATCH OUT!
        'xi': 1 / 100,
    },
    time_frame=TimeFrame.of("2015-01-01", 150),
    initializer=partial(single_location, location=1, seed_size=5),
    rng=default_rng(1)
)

with sim_messaging(sim):
    out = sim.run()

Running simulation (StandardSimulation):
• 2015-01-01 to 2015-05-31 (150 days)
• 6 geo nodes


IpmSimLessThanZeroException: 
Less than zero rate detected. When providing or defining ipm parameters, ensure that
they will not result in a negative rate. Note: this can often happen unintentionally
if a function is given as a parameter.

Showing current Node : Timestep
1: 0

Showing current compartment values
S: 9687648
I: 5
R: 0

Showing current ipm params
beta: 0.4
gamma: -0.2
xi: 0.01



## Test using function as a parameter

In [5]:
def run_and_plot(beta: ParamValue) -> None:
    sim = StandardSimulation(
        ipm=ipm_library['sirs'](),
        mm=mm_library['pei'](),
        geo=geo_library['pei'](),
        params={
            # IPM params
            'beta': beta,
            'gamma': 1 / 6,
            'xi': 1 / 90,
            # MM params
            'theta': 0.1,
            'move_control': 0.9,
        },
        time_frame=TimeFrame.of("2015-01-01", 150),
        initializer=partial(single_location, location=0, seed_size=10_000),
    )

    with sim_messaging(sim):
        sim.run()


def beta_fn(t, n):
    # NEGATIVE VALUES PRODUCED BY FUNCTION
    x = -0.35 + -0.05 * np.sin(-2 * np.pi * (t / dim.days))
    cutoff = 50 + (n * 3)
    if t > cutoff:
        pop = geo['population'][n]
        cut = 0.3 if pop < 9_000_000 else 0.25
        x -= cut
    return x


# The function is our beta value!
run_and_plot(beta_fn)

Running simulation (StandardSimulation):
• 2015-01-01 to 2015-05-31 (150 days)
• 6 geo nodes


IpmSimLessThanZeroException: 
Less than zero rate detected. When providing or defining ipm parameters, ensure that
they will not result in a negative rate. Note: this can often happen unintentionally
if a function is given as a parameter.

Showing current Node : Timestep
0: 0

Showing current compartment values
S: 18798973
I: 9988
R: 0

Showing current ipm params
beta: -0.35
gamma: 0.16666666666666666
xi: 0.011111111111111112



## Test with fork

In [6]:
from functools import partial

from epymorph import *
from epymorph.initializer import single_location

pei_geo = geo_library['pei']()

sim = StandardSimulation(
    geo=pei_geo,
    ipm=load_ipm_fork(),
    mm=mm_library['no'](),
    params={
        'phi': 40.0,
        'beta': -0.4,  # NEGATIVE NUMBER! WATCH OUT!
        'gamma': 1 / 5,
        'xi': 1 / 100,
        'prob1': 1 / 2,
        'prob2': 1 / 2
    },
    time_frame=TimeFrame.of("2015-01-01", 150),
    initializer=partial(single_location, location=1, seed_size=5),
    rng=default_rng(1)
)

with sim_messaging(sim):
    out = sim.run()

Running simulation (StandardSimulation):
• 2015-01-01 to 2015-05-31 (150 days)
• 6 geo nodes


IpmSimLessThanZeroException: 
Less than zero rate detected. When providing or defining ipm parameters, ensure that
they will not result in a negative rate. Note: this can often happen unintentionally
if a function is given as a parameter.

Showing current Node : Timestep
1: 0

Showing current compartment values
S: 9687648
I: 5
R: 0

Showing current ipm params
beta: -0.4
gamma: 0.2
xi: 0.01
prob1: 0.5
prob2: 0.5



# Case 2:

If we construct our geo to force a population at a node to be zero, we get the second error. (Note: this can also happen if a movement model moves everyone out of a node, so it's not as easy to fix as checking the geo before-hand.)

To handle this, the exception type `IpmSimNaNException` is used in `ipm_exec.py` to catch the divide by zero error, as it results in a NaN (not a number) rate.

In [7]:
from functools import partial

import numpy as np

from epymorph import *
from epymorph.geo.spec import NO_DURATION, AttribDef, StaticGeoSpec
from epymorph.geo.static import StaticGeo
from epymorph.initializer import single_location

my_geo = StaticGeo(
    spec=StaticGeoSpec(
        attributes=[
            AttribDef('label', dtype=str, shape=Shapes.N),
            AttribDef('population', dtype=str, shape=Shapes.N),
        ],
        time_period=NO_DURATION,
    ),
    values={
        'label': np.array(['a', 'b', 'c']),
        'population': np.array([0, 10, 20], dtype=np.int64),
    },
)


sim = StandardSimulation(
    geo=my_geo,
    ipm=load_ipm(),
    mm=mm_library['no'](),
    params={
        'phi': 40.0,
        'beta': 0.4,
        'gamma': 1 / 5,
        'xi': 1 / 90,
    },
    time_frame=TimeFrame.of("2015-01-01", 150),
    initializer=partial(single_location, location=1, seed_size=5),
    rng=default_rng(1)
)

with sim_messaging(sim):
    out = sim.run()

Running simulation (StandardSimulation):
• 2015-01-01 to 2015-05-31 (150 days)
• 3 geo nodes


IpmSimNaNException: 
NaN (not a number) rate detected. This is often the result of a divide by zero error.
When constructing the IPM, ensure that no edge transitions can result in division by zero
This commonly occurs when defining an S->I edge that is (some rate / sum of the compartments)
To fix this, change the edge to define the S->I edge as (some rate / Max(1/sum of the the compartments))
See examples of this in the provided example ipm definitions in the data/ipms folder.

Showing current Node : Timestep
0: 0

Showing current compartment values
S: 0
I: 0
R: 0

Showing current ipm params
beta: 0.4
gamma: 0.2
xi: 0.011111111111111112

Showing current corresponding transition
S->I: I*S*beta/(I + R + S)



## test with fork

In [8]:

from functools import partial

import numpy as np

from epymorph import *
from epymorph.geo.spec import NO_DURATION, AttribDef, StaticGeoSpec
from epymorph.geo.static import StaticGeo
from epymorph.initializer import single_location

my_geo = StaticGeo(
    spec=StaticGeoSpec(
        attributes=[
            AttribDef('label', dtype=str, shape=Shapes.N),
            AttribDef('population', dtype=str, shape=Shapes.N),
        ],
        time_period=NO_DURATION,
    ),
    values={
        'label': np.array(['a', 'b', 'c']),
        'population': np.array([0, 10, 20], dtype=np.int64),
    },
)


sim = StandardSimulation(
    geo=my_geo,
    ipm=load_ipm_fork(),
    mm=mm_library['no'](),
    params={
        'phi': 40.0,
        'beta': 0.4,
        'gamma': 1 / 5,
        'xi': 1 / 90,
        'prob1': 1 / 2,
        'prob2': 1 / 2
    },
    time_frame=TimeFrame.of("2015-01-01", 150),
    initializer=partial(single_location, location=1, seed_size=5),
    rng=default_rng(1)
)

with sim_messaging(sim):
    out = sim.run()

Running simulation (StandardSimulation):
• 2015-01-01 to 2015-05-31 (150 days)
• 3 geo nodes


IpmSimNaNException: 
NaN (not a number) rate detected. This is often the result of a divide by zero error.
When constructing the IPM, ensure that no edge transitions can result in division by zero
This commonly occurs when defining an S->I edge that is (some rate / sum of the compartments)
To fix this, change the edge to define the S->I edge as (some rate / Max(1/sum of the the compartments))
See examples of this in the provided example ipm definitions in the data/ipms folder.

Showing current Node : Timestep
0: 0

Showing current compartment values
S: 0
I: 0
R: 0

Showing current ipm params
beta: 0.4
gamma: 0.2
xi: 0.011111111111111112
prob1: 0.5
prob2: 0.5

Showing current corresponding fork transition
S->(I, R): I*S*beta*(prob1 + prob2)/(I + R + S)



## Case 3:

If we just provide a negative value to `prob1`, which is used in the demo fork ipm, we get a negative probability

To handle this, the exception type `IpmSimInvalidProbsException` is used in `ipm_exec.py` to catch the less than zero error.

In [9]:
sim = StandardSimulation(
    geo=pei_geo,
    ipm=load_ipm_fork(),
    mm=mm_library['no'](),
    params={
        'beta': 0.4,
        'gamma': 1 / 5,
        'xi': 1 / 90,
        'prob1': -1 / 4,  # WATCHOUT: negative value
        'prob2': 1 / 5
    },
    time_frame=TimeFrame.of("2015-01-01", 150),
    initializer=partial(single_location, location=1, seed_size=5),
    rng=default_rng(1)
)

with sim_messaging(sim):
    out = sim.run()

Running simulation (StandardSimulation):
• 2015-01-01 to 2015-05-31 (150 days)
• 6 geo nodes


IpmSimInvalidProbsException: 
Invalid probabilities for fork definition detected. Probabilities for a 
given tick should always be nonnegative and sum to 1

Showing current Node : Timestep
0: 0

Showing current compartment values
S: 18811310
I: 0
R: 0

Showing current ipm params
beta: 0.4
gamma: 0.2
xi: 0.011111111111111112
prob1: -0.25
prob2: 0.2

Showing current corresponding fork transition and probabilities
S->(I, R): I*S*beta*(prob1 + prob2)/(I + R + S)
Probabilities: prob1/(prob1 + prob2), prob2/(prob1 + prob2)



In [12]:

sim = StandardSimulation(
    geo=geo_library['pei'](),
    ipm=ipm_library['sirh'](),
    mm=mm_library['no'](),
    params={
        'beta': 0.4,
        'gamma': 1 / 5,
        'xi': 1 / 100,
        'hospitalization_prob': -1 / 5,
        'hospitalization_duration': 15
    },
    time_frame=TimeFrame.of("2015-01-01", 150),
    initializer=partial(single_location, location=1, seed_size=5),
    rng=default_rng(1)
)

sim.run()

IpmSimInvalidProbsException: 
Invalid probabilities for fork definition detected. Probabilities for a 
given tick should always be nonnegative and sum to 1

Showing current Node : Timestep
0: 0

Showing current compartment values
S: 18811310
I: 0
R: 0
H: 0

Showing current ipm params
beta: 0.4
gamma: 0.2
xi: 0.01
hospitalization_prob: -0.2
hospitalization_duration: 15.0

Showing current corresponding fork transition and probabilities
I->(H, R): I*gamma
Probabilities: hospitalization_prob, 1 - hospitalization_prob



# Extending this work
Overall, this analysis has resulted in error handling for common ipm simulation errors. However, if more are to arise in the future, they can be expanded in the `epymorph/error.py` module
by inhereting from `IpmSimException`, or `IpmSimExceptionWithFields` if simulation values are to be printed along with the error. See the definition of `IpmSimNanException` in `epymorph/error.py` for an example.